#  Testing through documentation

26.3 doctest — Test interactive Python examples   
https://docs.python.org/3/library/doctest.html


<b>doctest</b> lets you <b>test</b> your code by running <b>examples embedded in the documentation</b> and verifying that they produce the expected results. 

It works by parsing the help text to find examples, running them, then comparing the output text against the expected value. 

Many developers find `doctest` **easier** than `unittest` because in its simplest form, there is no API to learn before using it.

However, as the examples become more complex **the lack of fixture management** can make writing doctest tests more **cumbersome** than using unittest.

## 1 Getting Started

**doctest** looks for `lines` 

* **beginning** with ：

   * **>>>** the `interpreter prompt` to find the beginning of a test case. 

* **ended** with 
      
    * a `blank` line</b>, 
      
    * the `next` interpreter prompt.

Here, `fun_multiply()` has two examples given in the module: ```doctest_simple.py```

In [ ]:
%%file ./code/doctest/doctest_simple.py

def fun_multiply(a, b):
    """
    >>> fun_multiply(2, 3)
    6
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

To **run** the tests, use

`doctest` as the `main program` via the **-m** option to the interpreter:

```bash
>python -m doctest *.py
```

In [ ]:
!python -m doctest ./code/doctest/doctest_simple.py

Usually **no output** is produced .It means **all the examples worked**

Pass ** -v ** to the script, and **doctest** prints a detailed log of what it’s trying, and prints a summary at the end:

```bash
>python -m doctest -v  *.py
```

In [ ]:
!python -m doctest -v ./code/doctest/doctest_simple.py

**Test Examples** cannot usually stand on their own as explanations of a function,

**doctest** also lets you keep the **surrounding text** you would normally include in the documentation. 

**Intervening text** is **ignored**, and can have `any format` as long as it does `not look like a test case`.



In [ ]:
%%file ./code/doctest/doctest_simple_with_docs.py

def fun_multiply(a, b):
    """Returns a * b.

    Works with numbers:
    
    >>> fun_multiply(2, 3)
    6

    and strings:
    
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

def fun_add(a, b):
    """Returns a + b.

    Works with numbers:
    
    >>> fun_add(2, 3)
    5
    
    and strings:
    
    >>> fun_add('1', '3')
    '13'
    """
    return a + b

In [ ]:
!python -m doctest -v ./code/doctest/doctest_simple_with_docs.py

The surrounding text in the updated `docstring `

* **useful to a human reader**,

* **ignored by doctest**, 

the results are the same.

# run doctest directly 

The simplest way to start using doctest  is to 

* **end** each module with:

```python
if __name__ == "__main__":
    import doctest
    doctest.testmod()
```

**doctest** then examines `docstrings` in the module .

In [ ]:
%%file  ./code/doctest/doctest_with_testmod.py

def fun_multiply(a, b):
    """Returns a * b.

    Works with numbers:
    
    >>> fun_multiply(2, 3)
    6

    and strings:
    
    >>> fun_multiply('a', 3)
    'aaa'
    """
    return a * b

def fun_add(a, b):
    """Returns a + b.

    Works with numbers:
    
    >>> fun_add(2, 3)
    5
    
    and strings:
    
    >>> fun_add('1', '3')
    '13'
    """
    return a+b
    #return a + b+1

if __name__ == "__main__":
    import doctest
    doctest.testmod()

In [ ]:
!python ./code/doctest/doctest_with_testmod.py

In [ ]:
!python ./code/doctest/doctest_with_testmod.py -v

## DocTest of iapws.iapws97

https://github.com/jjgomera/iapws


In [ ]:
!pip install iapws

In **Debian** you can find in **official repositories** in jessie. 

In **ubuntu** it's in official repositories from ubuntu saucy (13.10)

In [ ]:
!sudo apt install iapws

```python
# Boundary Region2-Region3
def _P23_T(T):
    """Define the boundary between Region 2 and 3, P=f(T)
    Parameters
    ----------
    T : float
        Temperature [K]
    Returns
    -------
    P : float
        Pressure [MPa]
    References
    ----------
    IAPWS, Revised Release on the IAPWS Industrial Formulation 1997 for the
    Thermodynamic Properties of Water and Steam August 2007,
    http://www.iapws.org/relguide/IF97-Rev.html, Eq 5
    Examples
    --------
    >>> _P23_T(623.15)
    16.52916425
    """
    n = [0.34805185628969e3, -0.11671859879975e1, 0.10192970039326e-2]
    return n[0]+n[1]*T+n[2]*T**2
```

In [ ]:
%%file ./code/doctest/doctest_iapws97.py

from iapws import iapws97

if __name__ == "__main__":
    import doctest
    doctest.testmod(iapws97)

### In Ubuntu,Python3

In [ ]:
!python3 ./code/doctest/doctest_iapws97.py

In [ ]:
!python3 ./code/doctest/doctest_iapws97.py -v

### under Windows 

In [ ]:
!python ./code/doctest/doctest_iapws97.py

## Further Reading

Python 3 Module of the Week¶

https://pymotw.com/3/

PyMOTW-3 is a series of articles written by `Doug Hellmann` to demonstrate how to use the modules of the Python 3 standard library. It is based on the original PyMOTW series, which covered Python 2.7. See About Python Module of the Week for details including the version of Python and tools used.


doctest – Testing through documentation:  

* https://pymotw.com/3/doctest/index.html